In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import sys
sys.path.append('../')

from utils.processing import *
from utils.visualisation import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, GridSearchCV, train_test_split
from sklearn.compose import TransformedTargetRegressor, make_column_transformer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder

np.random.seed(0)

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 80)

In [4]:
# load data set
df = pd.read_csv('../data/train.csv')
df_train, df_validate = train_test_split(df, test_size=0.30)

In [5]:
def drop_cols_for_regression(df):
    drop_columns = [
        'address',
        'title',
        'listing_id',
        'property_name',
        'total_num_units',
        'available_unit_types',
        'property_details_url',
        'elevation',
        'tenure',
        'property_type',
        'floor_level',
        'furnishing',
        'built_year',
    ]
    """
        'gep_pri_sch_within_1km',
        'gep_pri_sch_within_1km_2km',
        'gep_pri_sch_outside_2km',
        'pri_sch_within_500m',
        'pri_sch_outside_500m',
        'line_cc',
        'line_ce',
        'line_cg',
        'line_dt',
        'line_ew',
        'line_ne',
        'line_ns',
        'line_te',
        'furnishing_partial',
        'furnishing_unfurnished',
        'furnishing_unspecified',
    ]
    """
    return df.drop(columns=drop_columns)

def prepare_data_for_regression(df):
    adfs = read_aux_csv('../data')
    #merge_aux = dict((k, adfs[k]) for k in ['primary_schools', 'regions', 'mrt_stations'])
    merge_aux = dict((k, adfs[k]) for k in ['regions'])
    df = join_aux(df, merge_aux)
    df = drop_cols_for_regression(df)

    return df

In [6]:
#df = prepare_data_for_regression(df)
#df.head()
#df_train[df_train.isna().any(axis=1)]

df_train = preprocess(df_train)
df_train = prepare_data_for_regression(df_train)
df_train.head()

,num_beds,num_baths,size_sqft,lat,lng,subzone,planning_area,price,property_type_private,property_type_public,tenure_high_year,tenure_low_year,floor_level_ground,floor_level_high,floor_level_low,floor_level_mid,floor_level_penthouse,floor_level_top,furnishing_partial,furnishing_unfurnished,furnishing_unspecified,region
0,1.0,1.0,474,1.330709,103.868391,potong pasir,toa payoh,1060900.0,1,0,1,0,0,0,0,0,0,0,0,0,1,c
1,2.0,2.0,915,1.360919,103.892050,lorong ah soo,hougang,1029900.0,1,0,0,1,0,0,0,0,0,0,0,0,1,ne
2,4.0,4.0,4717,1.300620,103.892783,mountbatten,marine parade,8059800.0,1,0,1,0,0,0,0,0,0,0,0,0,1,c
3,3.0,1.0,699,1.316655,103.805632,farrer court,bukit timah,554400.0,0,1,0,1,0,0,0,0,0,0,0,0,1,c
4,3.0,3.0,1249,1.315795,103.832096,nassim,tanglin,3738000.0,1,0,1,0,0,0,0,0,0,0,0,0,1,c


In [7]:
X_train = df_train.drop('price', axis=1)
y_train = df_train['price']

In [8]:
numeric_features = ['size_sqft', 'num_beds', 'num_baths']
numeric_transformer = StandardScaler()

categorical_features = ["subzone", "planning_area", "region"]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = make_column_transformer(
    (numeric_transformer, numeric_features),
     (categorical_transformer, categorical_features))

In [9]:
def pipelined_model(model):
    pipe = make_pipeline(preprocessor, PolynomialFeatures(2), model)
    return TransformedTargetRegressor(regressor=pipe, transformer=StandardScaler())

In [10]:
def rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    score = np.sqrt(mse)
    return score
    
def run_models(models, X, y):
    def rmse_scorer():
        return make_scorer(rmse)
    
    model_scores = dict()
    
    for model in models:
        regr = pipelined_model(model)
        scores = cross_validate(regr, X, y, cv=10, scoring=rmse_scorer())
        regr.fit(X, y)
        model_scores[model.__class__.__name__] = [regr, scores]
        print(model.__class__.__name__, scores['test_score'].mean())
        
    return model_scores

In [11]:
models = [
            #KNeighborsRegressor(),
            Lasso(),
            Ridge(),
            #DecisionTreeRegressor(),
            #GradientBoostingRegressor(),
            #AdaBoostRegressor(),
            #RandomForestRegressor(max_depth=10),
            xgb.XGBRegressor(),
            #xgb.XGBRegressor(n_estimators=200),
            #xgb.XGBRegressor(max_depth=8, learning_rate=0.3, min_child_weight=1),
            #xgb.XGBRegressor(max_depth=8, learning_rate=0.3, min_child_weight=1, n_estimators=200)
        ]

model_scores = run_models(models, X_train, y_train)

Lasso 2700506.216840918
Ridge 1081697.8446429754
XGBRegressor 861037.1210314048


In [12]:
"""
import math
best_score = math.inf
best_model = None

for k, v in model_scores.items():
    regr, scores = v
    score = scores['test_score'].mean()
    
    print(k, score)
    
    if score < best_score:
        best_score = score
        best_model = regr
        
        
parameters = {'regressor__xgbregressor__max_depth':[1, 6, 8],
              #'regressor__xgbregressor__n_estimators':[100, 150, 200],
              'regressor__xgbregressor__min_child_weight': [1, 5, 10],
              'regressor__xgbregressor__learning_rate': [0.1, 0.3, 0.7],
             }

m = pipelined_model(xgb.XGBRegressor())
model = GridSearchCV(m, parameters)


model.fit(X_train, y_train)

best_params = model.best_params_
print(best_params)
"""

"\nimport math\nbest_score = math.inf\nbest_model = None\n\nfor k, v in model_scores.items():\n    regr, scores = v\n    score = scores['test_score'].mean()\n    \n    print(k, score)\n    \n    if score < best_score:\n        best_score = score\n        best_model = regr\n        \n        \nparameters = {'regressor__xgbregressor__max_depth':[1, 6, 8],\n              #'regressor__xgbregressor__n_estimators':[100, 150, 200],\n              'regressor__xgbregressor__min_child_weight': [1, 5, 10],\n              'regressor__xgbregressor__learning_rate': [0.1, 0.3, 0.7],\n             }\n\nm = pipelined_model(xgb.XGBRegressor())\nmodel = GridSearchCV(m, parameters)\n\n\nmodel.fit(X_train, y_train)\n\nbest_params = model.best_params_\nprint(best_params)\n"

In [13]:
y_validate = df_validate['price']
X_validate = prepare_data_for_regression(preprocess(df_validate.drop('price', axis=1), is_target=True, num_beds=df_train['num_beds'].median(), num_baths=df_train['num_baths'].median()))

for k, v in model_scores.items():
    regr, scores = v
    predict_validate = regr.predict(X_validate)

    print(k, rmse(y_validate, predict_validate))

Lasso 2628150748.646675
Ridge 2749741776.699389
XGBRegressor 2976770.856266274


In [14]:
# Retrain using best model
fmodel = pipelined_model(xgb.XGBRegressor())

df_ftrain = pd.read_csv('../data/train.csv')
df_ftrain = preprocess(df_ftrain)
df_ftrain = prepare_data_for_regression(df_ftrain)
df_ftrain.head()

X_ftrain = df_ftrain.drop('price', axis=1)
y_ftrain = df_ftrain['price']

fmodel.fit(X_ftrain, y_ftrain)
predict_ftrain = fmodel.predict(X_ftrain)
print(rmse(y_ftrain, predict_ftrain))

543434.1549984228


In [15]:
df_test = preprocess(pd.read_csv('../data/test.csv'), is_target=True, num_beds=df_train['num_beds'].median(), num_baths=df_train['num_baths'].median())

# further preprocessing
X_test = prepare_data_for_regression(df_test)

y_predict = fmodel.predict(X_test)

X_test['Predicted'] = y_predict

In [16]:
y_predict

array([1126885.5, 1509495.6, 1325369.2, ..., 3070837. , 1126024. ,
       3901163.5], dtype=float32)

In [17]:
submission = X_test[['Predicted']]
submission.to_csv('submission3.csv', index=True, index_label='id', header=True, columns=['Predicted'])